# Ponderada Tradução Automática
As etapas incluem:
 - Download e pré-processamento do conjunto de dados
 - Tokenização e construção dos vocabulários
 - Preparação dos dados (truncamento, preenchimento e criação do DataLoader)
 - Exibição de um minibatch como demonstração

In [1]:
!pip install d2l numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.3/98.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.0/125.0 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.0/95.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 39.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.3
    Uninstalling requests-2.32.3:
      Successfull

In [46]:
import os
import torch
from d2l import torch as d2l

### 1. Download e Pré-processamento do Conjunto de Dados

Configuramos o DATA_HUB com a URL do conjunto de dados e lemos o arquivo `fra.txt`.

In [47]:
d2l.DATA_HUB['fra-eng'] = (d2l.DATA_URL + 'fra-eng.zip',
                           '94646ad1522d915e7b0f9296181140edcf86a4f5')

def read_data_nmt():
    """
    Carrega o conjunto de dados inglês-francês.
    """
    data_dir = d2l.download_extract('fra-eng')
    with open(os.path.join(data_dir, 'fra.txt'), 'r') as f:
        return f.read()

raw_text = read_data_nmt()
print("Exemplo do texto bruto:")
print(raw_text[:75])

Exemplo do texto bruto:
Go.	Va !
Hi.	Salut !
Run!	Cours !
Run!	Courez !
Who?	Qui ?
Wow!	Ça alors !



In [48]:
def preprocess_nmt(text):
    """
    Pré-processa o conjunto de dados de tradução:
    - Substitui espaços não separáveis por espaços comuns.
    - Converte o texto para letras minúsculas.
    - Insere espaços entre palavras e pontuações (quando necessário).
    """
    def no_space(char, prev_char):
        return char in set(',.!?') and prev_char != ' '

    # Substitui e converte
    text = text.replace('\u202f', ' ').replace('\xa0', ' ').lower()

    # Insere espaço antes dos sinais de pontuação quando necessário
    out = [' ' + char if i > 0 and no_space(char, text[i - 1]) else char
           for i, char in enumerate(text)]
    return ''.join(out)

text = preprocess_nmt(raw_text)
print("\nExemplo do texto pré-processado:")
print(text[:80])


Exemplo do texto pré-processado:
go .	va !
hi .	salut !
run !	cours !
run !	courez !
who ?	qui ?
wow !	ça alors !


### 2. Tokenização e Construção do Vocabulário

Dividimos o texto em pares de frases (inglês e francês) e, em seguida, tokenizamos cada linha.

In [49]:
def tokenize_nmt(text, num_examples=None):
    """
    Tokeniza o conjunto de dados inglês-francês.
    Retorna duas listas de listas de tokens: uma para a fonte (inglês) e outra para o alvo (francês).
    """
    source, target = [], []
    for i, line in enumerate(text.split('\n')):
        if num_examples and i > num_examples:
            break
        parts = line.split('\t')
        if len(parts) == 2:
            source.append(parts[0].split(' '))
            target.append(parts[1].split(' '))
    return source, target

source, target = tokenize_nmt(text)
print("\nExemplo de tokenização:")
print("Fonte:", source[:6])
print("Alvo:  ", target[:6])


Exemplo de tokenização:
Fonte: [['go', '.'], ['hi', '.'], ['run', '!'], ['run', '!'], ['who', '?'], ['wow', '!']]
Alvo:   [['va', '!'], ['salut', '!'], ['cours', '!'], ['courez', '!'], ['qui', '?'], ['ça', 'alors', '!']]


### 3. Preparação dos Dados: Truncamento, Preenchimento e Conversão para Arrays

Criamos funções para garantir que todas as sequências tenham o mesmo comprimento (num_steps).

Adicionamos o token `<eos>` para indicar o fim da sequência e usamos o token `<pad>` para preencher as sequências curtas.


In [50]:
def truncate_pad(line, num_steps, padding_token):
    """
    Trunca ou preenche uma sequência para que ela tenha tamanho `num_steps`.
    """
    if len(line) > num_steps:
        return line[:num_steps]  # Trunca
    return line + [padding_token] * (num_steps - len(line))  # Preenche

In [51]:
def build_array_nmt(lines, vocab, num_steps):
    """
    Converte uma lista de sequências de tokens em um array:
    - Converte cada token em índice, adiciona o token <eos> ao final da sequência.
    - Trunca ou preenche cada sequência para que todas tenham o mesmo comprimento.
    - Calcula os comprimentos válidos (excluindo os tokens de preenchimento).
    """

    # Converte cada linha em índices
    lines = [vocab[l] for l in lines]

    # Adiciona o token de fim de sequência
    lines = [l + [vocab['<eos>']] for l in lines]
    array = torch.tensor([truncate_pad(l, num_steps, vocab['<pad>']) for l in lines])
    valid_len = (array != vocab['<pad>']).type(torch.int32).sum(1)
    return array, valid_len

### Criação dos vocabulários para os idiomas de origem e destino

In [52]:
# Criação dos vocabulários para os idiomas de origem e destino
src_vocab = d2l.Vocab(source, min_freq=2, reserved_tokens=['<pad>', '<bos>', '<eos>'])
tgt_vocab = d2l.Vocab(target, min_freq=2, reserved_tokens=['<pad>', '<bos>', '<eos>'])
print("Tamanho do vocabulário (fonte):", len(src_vocab))
print("Tamanho do vocabulário (alvo):", len(tgt_vocab))

Tamanho do vocabulário (fonte): 10012
Tamanho do vocabulário (alvo): 17851


### 4. Criação do DataLoader

Preparamos a função que retorna o iterador de dados, processando os pares de sequências com truncamento e preenchimento.


In [53]:
def load_data_nmt(batch_size, num_steps, num_examples=600):
    """
    Retorna:
    - data_iter: iterador de dados para treinamento em minibatches
    - src_vocab: vocabulário do idioma de origem
    - tgt_vocab: vocabulário do idioma de destino
    """
    # Reprocessa o texto completo (para garantir que usamos o mesmo pré-processamento)
    text = preprocess_nmt(read_data_nmt())
    source, target = tokenize_nmt(text, num_examples)
    src_vocab = d2l.Vocab(source, min_freq=2, reserved_tokens=['<pad>', '<bos>', '<eos>'])
    tgt_vocab = d2l.Vocab(target, min_freq=2, reserved_tokens=['<pad>', '<bos>', '<eos>'])
    src_array, src_valid_len = build_array_nmt(source, src_vocab, num_steps)
    tgt_array, tgt_valid_len = build_array_nmt(target, tgt_vocab, num_steps)
    data_arrays = (src_array, src_valid_len, tgt_array, tgt_valid_len)
    data_iter = d2l.load_array(data_arrays, batch_size)
    return data_iter, src_vocab, tgt_vocab

### Testa a criação do iterador com um minibatch de exemplo

In [54]:
# Testa a criação do iterador com um exemplo
batch_size = 2
num_steps = 8
train_iter, src_vocab, tgt_vocab = load_data_nmt(batch_size, num_steps)

for X, X_valid_len, Y, Y_valid_len in train_iter:
    print("\nMinibatch exemplo:")
    print('X:', X.type(torch.int32))
    print('Comprimentos válidos de X:', X_valid_len)
    print('Y:', Y.type(torch.int32))
    print('Comprimentos válidos de Y:', Y_valid_len)
    break



Minibatch exemplo:
X: tensor([[ 59, 138,   2,   4,   5,   5,   5,   5],
        [ 77,   6,   0,   4,   5,   5,   5,   5]], dtype=torch.int32)
Comprimentos válidos de X: tensor([4, 4])
Y: tensor([[14, 58,  0,  4,  5,  5,  5,  5],
        [46, 35,  6,  0,  4,  5,  5,  5]], dtype=torch.int32)
Comprimentos válidos de Y: tensor([4, 5])


In [55]:
num_examples_values = [100, 300, 600, 1000, 2000]

print("num_examples | Tamanho do vocabulário (origem) | Tamanho do vocabulário (destino)")
for num in num_examples_values:
    _, src_vocab, tgt_vocab = load_data_nmt(batch_size=2, num_steps=8, num_examples=num)
    print(f"{num:11d} | {len(src_vocab):28d} | {len(tgt_vocab):28d}")


num_examples | Tamanho do vocabulário (origem) | Tamanho do vocabulário (destino)
        100 |                           40 |                           40
        300 |                          102 |                          107
        600 |                          184 |                          201
       1000 |                          266 |                          321
       2000 |                          454 |                          585


In [56]:
!pip install jieba

In [57]:
import jieba

def tokenize_chinese(text):
    """
    Tokeniza um texto em chinês utilizando o jieba.
    """
    tokens = list(jieba.cut(text))
    return tokens

chinese_text = "测试塞缪尔" # "teste samuel"
tokens = tokenize_chinese(chinese_text)
print("Texto original:", chinese_text)
print("Tokens:", tokens)


Texto original: 测试塞缪尔
Tokens: ['测试', '塞缪尔']


In [58]:
tokens_simples = chinese_text.split(' ')
print("Tokenização simples:", tokens_simples)

Tokenização simples: ['测试塞缪尔']
